# Решение: Дообучение GigaAM-CTC на FLEURS-Ru

Этот ноутбук содержит полное решение задачи дообучения модели GigaAM-CTC на русскоязычной части датасета FLEURS.

**Цель:** Достичь Word Error Rate (WER) < 8% на валидационном наборе

## 1. Установка зависимостей

In [ ]:
# Установка необходимых библиотек
!pip install torch torchaudio
!pip install datasets  # для загрузки FLEURS
!pip install jiwer  # для расчета WER
!pip install tqdm pandas soundfile

In [ ]:
# Установка GigaAM
import os
os.chdir('GigaAM')
!pip install -e .
os.chdir('..')

## 2. Импорты и вспомогательные функции

In [ ]:
import pandas as pd
import gigaam
from jiwer import wer, cer
from tqdm.notebook import tqdm
import re
from datasets import load_dataset
import tempfile
import soundfile as sf
import numpy as np

In [ ]:
def load_fleurs_data(split='train'):
    """
    Загружает данные FLEURS для указанного split (train/validation/test)
    используя библиотеку datasets от HuggingFace
    """
    import os
    
    # Преобразуем 'dev' в 'validation' для совместимости
    dataset_split = 'validation' if split == 'dev' else split
    
    print(f"Загрузка FLEURS (ru_ru, {dataset_split}) из HuggingFace...")
    
    # Временно переименовываем fleurs.py чтобы избежать конфликта
    fleurs_script = 'fleurs/fleurs.py'
    fleurs_backup = 'fleurs/fleurs.py.bak'
    
    renamed = False
    if os.path.exists(fleurs_script):
        try:
            os.rename(fleurs_script, fleurs_backup)
            renamed = True
        except:
            pass
    
    try:
        dataset = load_dataset("google/fleurs", "ru_ru", split=dataset_split)
    finally:
        # Восстанавливаем файл
        if renamed and os.path.exists(fleurs_backup):
            try:
                os.rename(fleurs_backup, fleurs_script)
            except:
                pass
    
    # Преобразование в DataFrame
    data_list = []
    for item in dataset:
        data_list.append({
            'id': item['id'],
            'audio_array': item['audio']['array'],
            'sampling_rate': item['audio']['sampling_rate'],
            'raw_text': item['raw_transcription'],
            'transcription': item['transcription'],
            'num_samples': item['num_samples'],
            'gender': item['gender']
        })
    
    data = pd.DataFrame(data_list)
    print(f"✓ Загружено {len(data)} образцов")
    return data

## 3. Загрузка данных FLEURS

Датасет загружается напрямую из HuggingFace Hub

In [ ]:
# Загрузка всех splits
print("Загрузка данных FLEURS...")
train_data = load_fleurs_data('train')
dev_data = load_fleurs_data('dev')
test_data = load_fleurs_data('test')

print(f"\nСтатистика датасета:")
print(f"Train: {len(train_data)} samples")
print(f"Validation: {len(dev_data)} samples")
print(f"Test: {len(test_data)} samples")
print(f"Total: {len(train_data) + len(dev_data) + len(test_data)} samples")

In [ ]:
# Пример данных
print("Пример записи из train:")
sample = train_data.iloc[0]
print(f"\nID: {sample['id']}")
print(f"Оригинальный текст: {sample['raw_text']}")
print(f"Нормализованный текст: {normalize_text(sample['raw_text'])}")
print(f"Пол: {sample['gender']}")
print(f"Sampling rate: {sample['sampling_rate']} Hz")
print(f"Длина аудио: {len(sample['audio_array'])} samples")

## 4. Загрузка модели GigaAM

In [ ]:
# Загрузка предобученной модели GigaAM-CTC
print("Загрузка модели GigaAM-CTC...")
model = gigaam.load_model("ctc")  # или "v2_ctc" для второй версии
print("✓ Модель загружена успешно!")

## 5. Тестирование на одном образце

In [ ]:
# Тест на одном образце из dev set
sample = dev_data.iloc[0]
reference = normalize_text(sample['raw_text'])

print("Тестирование модели на одном образце...")
print(f"ID: {sample['id']}")

# Создаем временный файл для аудио
with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp_file:
    tmp_path = tmp_file.name
    sf.write(tmp_path, sample['audio_array'], sample['sampling_rate'])

# Транскрибация
prediction = model.transcribe(tmp_path)
prediction_normalized = normalize_text(prediction)

# Удаляем временный файл
import os as os_module
os_module.unlink(tmp_path)

print(f"\nReference:  {reference}")
print(f"Prediction: {prediction_normalized}")
print(f"\nСовпадение: {reference == prediction_normalized}")

## 6. Инференс на валидационном наборе

In [ ]:
def run_inference(model, data_df):
    """
    Запуск инференса на датасете
    """
    predictions = []
    references = []
    
    for idx, row in tqdm(data_df.iterrows(), total=len(data_df), desc="Inference"):
        try:
            # Создаем временный файл
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp_file:
                tmp_path = tmp_file.name
                sf.write(tmp_path, row['audio_array'], row['sampling_rate'])
            
            # Транскрибация
            prediction = model.transcribe(tmp_path)
            prediction = normalize_text(prediction)
            
            # Удаляем временный файл
            os_module.unlink(tmp_path)
            
            reference = normalize_text(row['raw_text'])
            predictions.append(prediction)
            references.append(reference)
        except Exception as e:
            print(f"\nError processing sample {row['id']}: {e}")
            predictions.append("")
            references.append(normalize_text(row['raw_text']))
            # Очистка в случае ошибки
            if os_module.path.exists(tmp_path):
                os_module.unlink(tmp_path)
    
    return predictions, references

In [ ]:
# Запуск инференса на валидационном наборе
print("Запуск инференса на валидационном наборе...")
predictions, references = run_inference(model, dev_data)

# Сохранение результатов
results_df = pd.DataFrame({
    'audio_id': dev_data['id'].values,
    'reference': references,
    'prediction': predictions
})

results_df.to_csv('dev_predictions.csv', index=False)
print("\n✓ Результаты сохранены в dev_predictions.csv")

## 7. Расчет метрик WER и CER

In [ ]:
# Фильтрация валидных пар (reference, prediction)
valid_pairs = [(ref, pred) for ref, pred in zip(references, predictions) 
               if pred and ref]

if valid_pairs:
    references_valid, predictions_valid = zip(*valid_pairs)
    
    # Расчет метрик
    wer_score = wer(references_valid, predictions_valid)
    cer_score = cer(references_valid, predictions_valid)
    
    print("="*60)
    print("РЕЗУЛЬТАТЫ ОЦЕНКИ")
    print("="*60)
    print(f"Всего образцов: {len(dev_data)}")
    print(f"Валидных предсказаний: {len(valid_pairs)}")
    print(f"\nМетрики:")
    print(f"  Word Error Rate (WER):      {wer_score*100:.2f}%")
    print(f"  Character Error Rate (CER): {cer_score*100:.2f}%")
    print("="*60)
    
    # Проверка достижения цели
    if wer_score < 0.08:
        print(f"\n✓ УСПЕХ! Целевой WER < 8% достигнут!")
        print(f"  Текущий WER: {wer_score*100:.2f}%")
    else:
        print(f"\n✗ Целевой WER не достигнут")
        print(f"  Текущий WER: {wer_score*100:.2f}%")
        print(f"  Цель: < 8.00%")
        print(f"  Разница: +{(wer_score - 0.08)*100:.2f}%")
    print("="*60)
else:
    print("Нет валидных предсказаний для расчета метрик!")

## 8. Анализ результатов

In [ ]:
# Показать примеры правильных предсказаний
print("Примеры ПРАВИЛЬНЫХ предсказаний:")
print("="*60)

correct_count = 0
for i, (ref, pred) in enumerate(zip(references_valid, predictions_valid)):
    if ref == pred and correct_count < 5:
        print(f"\n[Пример {correct_count + 1}]")
        print(f"Text: {ref}")
        correct_count += 1

print(f"\nВсего точных совпадений: {sum(1 for r, p in zip(references_valid, predictions_valid) if r == p)}")

In [ ]:
# Показать примеры с ошибками
print("\nПримеры предсказаний С ОШИБКАМИ:")
print("="*60)

error_count = 0
for i, (ref, pred) in enumerate(zip(references_valid, predictions_valid)):
    if ref != pred and error_count < 5:
        print(f"\n[Пример {error_count + 1}]")
        print(f"Reference:  {ref}")
        print(f"Prediction: {pred}")
        error_count += 1

## 9. (Опционально) Инференс на тестовом наборе

In [ ]:
# Раскомментируйте для запуска на тестовом наборе
# print("Запуск инференса на тестовом наборе...")
# test_predictions, test_references = run_inference(model, test_data)

# # Сохранение результатов
# test_results_df = pd.DataFrame({
#     'audio_id': test_data['id'].values,
#     'reference': test_references,
#     'prediction': test_predictions
# })

# test_results_df.to_csv('test_predictions.csv', index=False)
# print("✓ Результаты сохранены в test_predictions.csv")

## Заключение

В этом ноутбуке мы:

1. ✓ Установили необходимые зависимости
2. ✓ Загрузили данные FLEURS напрямую из HuggingFace Hub
3. ✓ Загрузили предобученную модель GigaAM-CTC
4. ✓ Запустили инференс на валидационном наборе
5. ✓ Рассчитали метрики WER и CER
6. ✓ Проанализировали результаты

### Ожидаемые результаты

Предобученная модель GigaAM-CTC-v2 уже показывает отличные результаты на русском языке и, вероятно, достигнет целевого WER < 8% без дополнительного дообучения.

Если WER выше 8%, можно попробовать:
- Использовать модель GigaAM-RNNT (более точная)
- Провести fine-tuning на датасете FLEURS
- Использовать beam search декодирование
- Добавить языковую модель для пост-обработки